In [1]:
import numpy as np
import cantera as ct

In [2]:
num_species = 53

# terget
J = np.zeros(num_species)

# properties from cantera
D = np.zeros(num_species)
D_bi = np.zeros((num_species, num_species))

# properties from table
q = np.zeros(num_species)

# properties obtained from (n, 6, 4) potential
mu = np.zeros(num_species)
mu_bi = np.zeros((num_species, num_species))

# state value of simulation
n = np.zeros(num_species)

# parameter of simulation
E = 100

In [ ]:
# (n, 6, 4) potential
def phi_n64(r):

    # parameters
    K1 = 1.767
    K2 = 0.72
    kappa = 0.095

    # (19)
    C6_i = np.zeros(num_species)
    for i in range(num_species):
        C6_i[i] = 3*np.pi*epsilon_0*alpha[i]**2*I_e[i]

    # (20)
    alpha_q[i] = 2*C6_i[i]/e**2

    # (17)
    xi = np.zeros((num_species, num_species))
    for i in range(num_species):
        for j in range(num_species):
            xi[i, j] = alpha[i]/(q[i]**2*(1 + (2*alpha[i]/alpha[j])**(2/3))*alpha[j]**0.5)

    # (18)
    sigma = np.zeros((num_species, num_species))
    for i in range(num_species):
        for j in range(num_species):
            sigma[i, j] = K1*(alpha[i]**(1/3) + alpha[j]**(1/3))/(alpha[i]*alpha[j]*(1 + 1/xi[i,j]))**kappa

    # (16)
    C6 = np.zeros((num_species, num_species))
    for i in range(num_species):
        for j in range(num_species):
            C6[i, j] = 2*C6_i[i]*C6_i[j]/((alpha[j]/alpha[i])*C6_i[i] + (alpha[i]/alpha[j])*C6_i[j])

    # (15)
    epsilon = np.zeros((num_species, num_species))
    for i in range(num_species):
        for j in range(num_species):
            epsilon[i, j] = K2*(alpha[j]*q[i]**2*e**2/(8*np.pi*epsilon_0*sigma[i, j]**4))*(1 + xi[i, j])

    # (14)
    gamma = np.zeros((num_species, num_species))
    for i in range(num_species):
        for j in range(num_species):
            gamma[i, j] = (2/q[i]**2/(C6[i, j]/e**2) * alpha[q, j])/(alpha[j]*sigma[i, j]**2)

    # (13)
    phi_0 = N*epsilon[i,j]/(N*(3 + gamma[i,j] - 12*(1 + gamma[i,j])))

    # (12)
    phi = phi_0*(
        12/N*(1+gamma[i,j])*(sigma[i, j]/r)**n
        + 4*gamma[i, j]*(sigma[i, j]/r)**6
        - 3*(1 - gamma[i, j])(sigma[i, j]/r)**4
        )


    return phi

In [ ]:
i = 0
J[i] = -D[i]*Nabla(n[i]) + q[i]/np.abs(q[i])*mu[i]*E*n[i]